### Done


### ToDo
* break training up in smaller batches!
* make load also load self.frame and self.episode

# CartPole with Policy Gradient

In [ ]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',

    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }
});

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

## Standard libraries
import time

## 3rd party libraries
import tensorflow as tf
import numpy as np
# %matplotlib inline
# import matplotlib.pyplot0as plt
# import numpy as np
import gym

## Custom libraries
import modelPolicyGradient
import utils
import CartPole_config as config

In [ ]:
## Settings and parameters
experiment_name = None
# experiment_name = '2017-07-13-(05-16-25)' # Complete shit
# experiment_name = '2017-07-13-(05-59-03)' # Complete shit
# experiment_name = '2017-07-12-(17-09-11)' # Almost shit


## Training settings
learning_rate = 1e-3
max_train_frame = 2e6

In [ ]:
## Derived settings
run_name = experiment_name or utils.time_str()
logdir = './logdir/'+config.env_name+'/PG/' + run_name
print('logdir\t\t', logdir)

In [ ]:
num_run = 1
env = gym.make(config.env_name)
for i in range(num_run):
    run_name = experiment_name or utils.time_str()
    logdir = './logdir/'+config.env_name+'/PG/' + run_name
    print('logdir\t\t', logdir)

    ## Setup
    tf.reset_default_graph()
    random_seed = int(time.time())
    tf.set_random_seed(random_seed)
    np.random.seed(random_seed)
    print('random seed\t', random_seed)

    ## Build model
    agent = modelPolicyGradient.PolicyGradient(config=config, env=env,
                  learning_rate=learning_rate, logdir=logdir, 
                  max_train_frame=max_train_frame,  render=False)
    agent.build()
    agent.run(load_model=True)
    print('done')
    print('\n\n')

In [ ]:
logdir

In [ ]:
## Create a video of the results

if 0:
    tf.reset_default_graph()
    agent = modelPolicyGradient.PolicyGradient(config=config,  
                  learning_rate=learning_rate, logdir=logdir, 
                  max_train_frame=max_train_frame,  render=False)
    agent.build()
    agent.load_model(logdir)


    for i in range(10):
        target='./tmp/' + run_name + '_' + str(i) + '.gif'
        agent.create_video(run_name, target, num_episodes=1)

### WARNING: gif gets distorted in Jupyter!
# from IPython.display import Image
# Image(url=target)

print('done')

In [ ]:
ckpt = tf.train.get_checkpoint_state(logdir)

agent.saver.restore(agent.sess, ckpt.model_checkpoint_path)

In [ ]:
1